In [1]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import os
import warnings
warnings.filterwarnings('ignore')

In [22]:
# Load the dataset into a DataFrame
data = pd.read_csv('C:/Users/tam05/downloads/Loan Prediction Problem Dataset/train_u6lujuX_CVtuZ9i.csv')
data.head()

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LP001002,Male,No,0,Graduate,No,5849,0.0,NaN,360.0,1.0,Urban,Y
1,LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N
2,LP001005,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y
3,LP001006,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y
4,LP001008,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y


In [23]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 614 entries, 0 to 613
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Loan_ID            614 non-null    object 
 1   Gender             601 non-null    object 
 2   Married            611 non-null    object 
 3   Dependents         599 non-null    object 
 4   Education          614 non-null    object 
 5   Self_Employed      582 non-null    object 
 6   ApplicantIncome    614 non-null    int64  
 7   CoapplicantIncome  614 non-null    float64
 8   LoanAmount         592 non-null    float64
 9   Loan_Amount_Term   600 non-null    float64
 10  Credit_History     564 non-null    float64
 11  Property_Area      614 non-null    object 
 12  Loan_Status        614 non-null    object 
dtypes: float64(4), int64(1), object(8)
memory usage: 62.5+ KB


In [24]:
data.isnull().sum().sort_values(ascending=False)

Credit_History       50
Self_Employed        32
LoanAmount           22
Dependents           15
Loan_Amount_Term     14
Gender               13
Married               3
Loan_ID               0
Education             0
ApplicantIncome       0
CoapplicantIncome     0
Property_Area         0
Loan_Status           0
dtype: int64

In [25]:
#for numerical type
data ['LoanAmount'] = data ['LoanAmount'].fillna(data ['LoanAmount'].mean())
data ['Loan_Amount_Term'] = data ['Loan_Amount_Term'].fillna(data ['Loan_Amount_Term'].mean())
data ['Credit_History'] = data ['Credit_History'].fillna(data ['Credit_History'].mean())

In [26]:
#For ccategorical type 
data ['Gender'] = data ['Gender'].fillna(data['Gender'].mode()[0])
data ['Married'] = data ['Married'].fillna(data['Married'].mode()[0])
data ['Dependents'] = data ['Dependents'].fillna(data['Dependents'].mode()[0])
data ['Self_Employed'] = data ['Self_Employed'].fillna(data['Self_Employed'].mode()[0])

In [27]:
data.isnull().sum()

Loan_ID              0
Gender               0
Married              0
Dependents           0
Education            0
Self_Employed        0
ApplicantIncome      0
CoapplicantIncome    0
LoanAmount           0
Loan_Amount_Term     0
Credit_History       0
Property_Area        0
Loan_Status          0
dtype: int64

In [28]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 614 entries, 0 to 613
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Loan_ID            614 non-null    object 
 1   Gender             614 non-null    object 
 2   Married            614 non-null    object 
 3   Dependents         614 non-null    object 
 4   Education          614 non-null    object 
 5   Self_Employed      614 non-null    object 
 6   ApplicantIncome    614 non-null    int64  
 7   CoapplicantIncome  614 non-null    float64
 8   LoanAmount         614 non-null    float64
 9   Loan_Amount_Term   614 non-null    float64
 10  Credit_History     614 non-null    float64
 11  Property_Area      614 non-null    object 
 12  Loan_Status        614 non-null    object 
dtypes: float64(4), int64(1), object(8)
memory usage: 62.5+ KB


In [29]:
weights = {
    'ApplicantIncome': 0.3,
    'Credit_History': 0.4,
    'Property_Area_Rural': 0.1,
    'Property_Area_Semiurban': 0.1,
    'Property_Area_Urban': 0.1,
    'Education': 0.1
}

In [30]:
# Normalize the weights
total_weight = sum(weights.values())
weights = {k: v/total_weight for k, v in weights.items()}

# Convert the Property_Area column to dummy variables
data = pd.get_dummies(data, columns=['Property_Area'], drop_first=True)

In [31]:
# Calculate the content weight score for each row
data['content_weight'] = (data['ApplicantIncome']*weights['ApplicantIncome'] +
                        data['Credit_History']*weights['Credit_History'] +
                        data['Property_Area_Rural']*weights['Property_Area_Rural'] +
                        data['Property_Area_Semiurban']*weights['Property_Area_Semiurban'] +
                        data['Property_Area_Urban']*weights['Property_Area_Urban'] +
                        data['Education']*weights['Education'])

KeyError: 'Property_Area_Rural'

In [ ]:
#for gender
sns.countplot(x='Gender',data=data)
#as we can see Male applicant is more as compared to Female

In [ ]:
#How do women and men compare when it comes to defaulting on loans in the historical dataset?
comp=data[['Loan_Status','Gender']].value_counts().to_frame()
comp

In [ ]:
sns.countplot(x='Married',data=data)

In [ ]:
sns.countplot(x='Dependents',data=data)

In [ ]:
sns.countplot(x='Education',data=data)

In [ ]:
sns.countplot(x='Self_Employed',data=data)

In [ ]:
sns.countplot(x='Property_Area',data=data)

In [ ]:
data['ApplicantIncome']= np.log(data['ApplicantIncome'])
data['CoapplicantIncome']= np.log(data['CoapplicantIncome']+1)
data['LoanAmount'] = np.log(data['LoanAmount'])
data['Loan_Amount_Term']= np.log(data['Loan_Amount_Term'])

In [ ]:
sns.displot(data['ApplicantIncome'])

In [ ]:
sns.displot(data['CoapplicantIncome'])

In [ ]:
sns.histplot(data['LoanAmount'])

In [ ]:
sns.histplot(data['Loan_Amount_Term'])

In [ ]:
sns.displot(data['Credit_History'])

In [ ]:
corr=data.corr()
plt.figure(figsize=(12,7))
sns.heatmap(corr,annot=True, cmap='RdBu')

In [ ]:
data

In [ ]:
data_to_csv = data.drop(['Loan_ID','Loan_Status'], axis=1)
data_to_csv

In [ ]:
# Preprocess the data
data1 = pd.get_dummies(data, drop_first=True)

In [ ]:
X = data.drop(['Loan_ID', 'Loan_Status', 'content_weight'], axis=1)
y = data['Loan_Status']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

clf = DecisionTreeClassifier(random_state=42)
clf.fit(X_train, y_train, sample_weight=X_train['content_weight'])

# Evaluate the model performance
y_pred = clf.predict(X_test)


score = f1_score(y_test, y_pred, average='binary', sample_weight=X_test['content_weight'])
print('F1-score:', score)

In [ ]:
import joblib
joblib.dump(clf, 'loan_prediction_model.pkl')

In [ ]:
# Load the decision tree model from file
clf = joblib.load('loan_prediction_model.pkl')

# Load the column names for the feature variables
columns = pd.read_csv('C:/Users/tam05/downloads/Loan Prediction Problem Dataset/train_u6lujuX_CVtuZ9i.csv', nrows=0).columns[1:-1]
columns

In [ ]:
# Define a function to get user input
def get_input():
    inputs = {}
    for col in columns:
        val = input("Enter {}: ".format(col))
        if col in ['ApplicantIncome', 'CoapplicantIncome', 'LoanAmount', 'Loan_Amount_Term']:
            val = float(val)
        elif col == 'Credit_History':
            val = int(val)
        inputs[col] = [val]
    df = pd.DataFrame(inputs)
    # Add missing columns with default values
    for col in X.columns:
        if col not in df.columns:
            df[col] = 0
    # Reorder columns to match training dataset
    df = df[X.columns]
    return df


# Define a function to predict loan approval status
def predict_loan_approval(inputs):
    # Preprocess the input data
    inputs = pd.get_dummies(inputs, drop_first=True)
#     print(inputs)
    # Make predictions using the decision tree model
    y_pred = clf.predict(inputs)
    # Get the reasons for the loan approval decision
    reasons = []
    if y_pred[0] == 0:
        reasons.append('Customer does not meet credit history requirement')
    if y_pred[0] == 1:
        reasons.append('Customer does meet credit history requirement')
    if inputs['ApplicantIncome'].values[0] + inputs['CoapplicantIncome'].values[0] < 5000 and y_pred[0] == 0:
        reasons.append('Customer does not meet income requirement')
    if inputs['ApplicantIncome'].values[0] + inputs['CoapplicantIncome'].values[0] > 5000 and y_pred[0] == 1:
        reasons.append('Customer does meet income requirement')
    if inputs['LoanAmount'].values[0] > 150 and y_pred[0] == 0:
        reasons.append('Customer exceeds loan amount requirement')
    if inputs['Property_Area_Semiurban'].values[0] == 0 and y_pred[0] == 0:
        reasons.append('Customer does not meet property area requirement')
    if inputs['Property_Area_Semiurban'].values[0] == 1 and y_pred[0] == 1:
        reasons.append('Customer does meet property area requirement')    
    return y_pred[0], reasons

# Get user input and make loan approval decision
inputs = get_input()
approval_status, reasons = predict_loan_approval(inputs)

reasons_sentence = ", ".join(reasons)

print("======================================================")
# Print the loan approval decision and reasons
if approval_status == 1:
    print("Loan application approved.")
else:
    print("Loan application denied.")
print("Reasons: {}.".format(reasons_sentence))

In [ ]:
inputs